In [2]:
import numpy as np

def sigmoid(t):
    """apply sigmoid function on t.

    Args:
        t: scalar or numpy array

    Returns:
        scalar or numpy array

    >>> sigmoid(np.array([0.1]))
    array([0.52497919])
    >>> sigmoid(np.array([0.1, 0.1]))
    array([0.52497919, 0.52497919])
    """
    
    return np.e**t/(1+np.e**t)

In [20]:
def penalized_logistic_regression(y, tx, w, lambda_):
    """return the loss and gradient.

    Args:
        y:  shape=(N, 1)
        tx: shape=(N, D)
        w:  shape=(D, 1)
        lambda_: scalar

    Returns:
        loss: scalar number
        gradient: shape=(D, 1)

    >>> y = np.c_[[0., 1.]]
    >>> tx = np.arange(6).reshape(2, 3)
    >>> w = np.array([[0.1], [0.2], [0.3]])
    >>> lambda_ = 0.1
    >>> loss, gradient = penalized_logistic_regression(y, tx, w, lambda_)
    >>> round(loss, 8)
    0.62137268
    >>> gradient
    array([[-0.08370763],
           [ 0.2467104 ],
           [ 0.57712843]])
    """
    
    # Calculate log-loss L
    
    # New vectorized loss computation
    sig_pred = sigmoid(tx @ w)
    loss = -1 / len(y) * (y.T @ np.log(sig_pred) + (1 - y).T @ np.log(1 - sig_pred)).item() + (lambda_ / 2) * (w[1:].T @ w[1:]).item()
    
    
    # Gradient with regularization (excluding bias term)
    reg_term = np.copy(w)
    reg_term[0] = 0  # No regularization for the bias term
    gradient = 1 / len(y) * tx.T @ (sigmoid(tx @ w) - y) + lambda_ * reg_term
    
    #Calculating the hessian
    #Creating S matrix with sigma values in diagonale
    sigmas = []
    for xi in tx:
        sig_xi = sigmoid(xi.T @ w)
        temp = sig_xi * (1 - sig_xi)
        sigmas.append(temp.item())
    
    # Matrix with sig (1- sig) in diagonale
    S = np.diag(sigmas)
    hess_regul = np.identity(len(tx[0]))
    hess_regul[0, 0] = 0 # Not regularizing the w0
    hessian =  1 / len(y) * tx.T @ S @ tx + hess_regul
    
    
    return loss, gradient, hessian

def log_learning_by_penalized_gradient(y, tx, w, gamma, lambda_, hessian_w = False):
    """
    Do one step of gradient descent, using the penalized logistic regression.
    Return the loss and updated w.

    Args:
        y:  shape=(N, 1)
        tx: shape=(N, D)
        w:  shape=(D, 1)
        gamma: scalar
        lambda_: scalar

    Returns:
        loss: scalar number
        w: shape=(D, 1)

    >>> np.set_printoptions(8)
    >>> y = np.c_[[0., 1.]]
    >>> tx = np.arange(6).reshape(2, 3)
    >>> w = np.array([[0.1], [0.2], [0.3]])
    >>> lambda_ = 0.1
    >>> gamma = 0.1
    >>> loss, w = learning_by_penalized_gradient(y, tx, w, gamma, lambda_)
    >>> round(loss, 8)
    0.62137268
    >>> w
    array([[0.10837076],
           [0.17532896],
           [0.24228716]])
    """
    
    loss, gradient, hessian = penalized_logistic_regression(y, tx, w, lambda_)
    if hessian_w == True:
        w -= gamma * np.linalg.inv(hessian) @ gradient
    else:
        w -= gamma * gradient
    
    return w, loss

def reg_logistic_regression(y, tx, lambda_, initial_w, max_iters, gamma):
    """
    Do logistic regression with regulatization until the threshold or max_iter is reached.
    
    Args:
        y:            shape=(N, 1)
        tx:           shape=(N, D+1)
        initial_w:    shape=(D+1, 1)
        max_iter:     int
        gamma:        float
        
    Returns: 
        w:         shape=(D+1, 1)
        loss:      loss at final w
        
    Example:
    w_final, losses = reg_logistic_regression(y, tx, initial_w, max_iter, gamma)
    
    
    """
    # init parameters
    threshold = 1e-8
    losses = []

    # build w
    w = initial_w

    # start the logistic regression
    for iter in range(max_iters):
        # get loss and update w.
        w, loss = learning_by_penalized_gradient(y.reshape(-1, 1), tx, w, gamma, lambda_)
        # converge criterion
        losses.append(loss)
        if len(losses) > 1 and np.abs(losses[-1] - losses[-2]) < threshold:
            break
    
    return w, loss

In [19]:
# Generate sample data
tx = np.array([[1, 2], [2, 3], [3, 4], [4, 5], [5, 6]])  # Input features
y_binary = np.array([0, 0, 0, 1, 1])  # Reshape to (5, 1)
initial_w_log = np.zeros((tx.shape[1], 1))  # Initial weights for logistic regression (shape (D,1))
lambda_ = 0.05
max_iters = 10000  # Maximum iterations
gamma = 0.01  # Learning rate
lambda_ = 0.001  # Regularization parameter

# Test Logistic Regression
w_log, loss_log = reg_logistic_regression(y_binary, tx, lambda_, initial_w_log, max_iters, gamma)
print("Logistic Regression - Loss:", loss_log)
print("Logistic Regression - Weights:", w_log)

Logistic Regression - Loss: 0.27624428668768497
Logistic Regression - Weights: [[ 4.77107348]
 [-3.65488174]]
